In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import data_request as dr

In [15]:
df = dr.retrieve_data_request()
df

,output variable name,units,ag,long_name,standard_name,priority,comments,frequency
0,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,mon
1,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,day
2,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,1hr
3,tasmax,K,NaN,Daily Maximum Near-Surface Air Temperature,air_temperature,CORE,maximum from all integrated time steps per day,mon
4,tasmax,K,NaN,Daily Maximum Near-Surface Air Temperature,air_temperature,CORE,maximum from all integrated time steps per day,day
...,...,...,...,...,...,...,...,...
582,rootd,m,NaN,Maximum Root Depth,root_depth,TIER 2,NaN,fx
583,sftlaf,%,NaN,Percentage of the Grid Cell Occupied by Lake,lake_area_fraction,TIER 2,not in CMIP or in CF,fx
584,sfturf,%,NaN,Percentage of the Grid Cell Occupied by City,urban_area_fraction,TIER 2,not in CMIP or in CF,fx
585,dtb,m,NaN,Depth to Bedrock,bedrock_depth,TIER 2,not in CMIP or in CF (lower boundary of land s...,fx


In [4]:
df.groupby(["priority", "output variable name"])["frequency"].apply(list)

priority  output variable name
CORE      clt                     [mon, day, 1hr]
          evspsbl                 [mon, day, 1hr]
          hurs                    [mon, day, 1hr]
          huss                    [mon, day, 1hr]
          orog                               [fx]
                                       ...       
TIER 2    zg20                    [mon, day, 6hr]
          zg30                    [mon, day, 6hr]
          zg50                    [mon, day, 6hr]
          zg70                    [mon, day, 6hr]
          zg750                   [mon, day, 6hr]
Name: frequency, Length: 199, dtype: object

In [5]:
df.to_csv("cordex-cmip6-data-request.csv", index=False)

In [6]:
dr.get_cmip6_cmor_table("3hr")["variable_entry"].keys()

dict_keys(['clt', 'hfls', 'hfss', 'huss', 'mrro', 'mrsos', 'pr', 'prc', 'prsn', 'ps', 'rlds', 'rldscs', 'rlus', 'rsds', 'rsdscs', 'rsdsdiff', 'rsus', 'rsuscs', 'tas', 'tos', 'tslsi', 'uas', 'vas'])

In [7]:
data = dr.get_variable_entries_by_attributes(
    table="Amon", long_name="Near-Surface Air Temperature"
)  # .keys()
data

{'tas': {'frequency': 'mon',
  'modeling_realm': 'atmos',
  'standard_name': 'air_temperature',
  'units': 'K',
  'cell_methods': 'area: time: mean',
  'cell_measures': 'area: areacella',
  'long_name': 'Near-Surface Air Temperature',
  'comment': 'near-surface (usually, 2 meter) air temperature',
  'dimensions': 'longitude latitude time height2m',
  'out_name': 'tas',
  'type': 'real',
  'positive': '',
  'valid_min': '',
  'valid_max': '',
  'ok_min_mean_abs': '',
  'ok_max_mean_abs': ''}}

In [8]:
data = dr.get_variable_entries_by_attributes(
    table="Amon", standard_name="air_temperature"
)  # .keys()
data

{'ta': {'frequency': 'mon',
  'modeling_realm': 'atmos',
  'standard_name': 'air_temperature',
  'units': 'K',
  'cell_methods': 'time: mean',
  'cell_measures': 'area: areacella',
  'long_name': 'Air Temperature',
  'comment': 'Air Temperature',
  'dimensions': 'longitude latitude plev19 time',
  'out_name': 'ta',
  'type': 'real',
  'positive': '',
  'valid_min': '',
  'valid_max': '',
  'ok_min_mean_abs': '',
  'ok_max_mean_abs': ''},
 'tas': {'frequency': 'mon',
  'modeling_realm': 'atmos',
  'standard_name': 'air_temperature',
  'units': 'K',
  'cell_methods': 'area: time: mean',
  'cell_measures': 'area: areacella',
  'long_name': 'Near-Surface Air Temperature',
  'comment': 'near-surface (usually, 2 meter) air temperature',
  'dimensions': 'longitude latitude time height2m',
  'out_name': 'tas',
  'type': 'real',
  'positive': '',
  'valid_min': '',
  'valid_max': '',
  'ok_min_mean_abs': '',
  'ok_max_mean_abs': ''},
 'tasmax': {'frequency': 'mon',
  'modeling_realm': 'atmos',


In [9]:
dr.cmip6_table_list("variables")

['CMIP6_3hr.json',
 'CMIP6_6hrLev.json',
 'CMIP6_6hrPlev.json',
 'CMIP6_6hrPlevPt.json',
 'CMIP6_AERday.json',
 'CMIP6_AERhr.json',
 'CMIP6_AERmon.json',
 'CMIP6_AERmonZ.json',
 'CMIP6_Amon.json',
 'CMIP6_CF3hr.json',
 'CMIP6_CFday.json',
 'CMIP6_CFmon.json',
 'CMIP6_CFsubhr.json',
 'CMIP6_E1hr.json',
 'CMIP6_E1hrClimMon.json',
 'CMIP6_E3hr.json',
 'CMIP6_E3hrPt.json',
 'CMIP6_E6hrZ.json',
 'CMIP6_Eday.json',
 'CMIP6_EdayZ.json',
 'CMIP6_Efx.json',
 'CMIP6_Emon.json',
 'CMIP6_EmonZ.json',
 'CMIP6_Esubhr.json',
 'CMIP6_Eyr.json',
 'CMIP6_IfxAnt.json',
 'CMIP6_IfxGre.json',
 'CMIP6_ImonAnt.json',
 'CMIP6_ImonGre.json',
 'CMIP6_IyrAnt.json',
 'CMIP6_IyrGre.json',
 'CMIP6_LImon.json',
 'CMIP6_Lmon.json',
 'CMIP6_Oclim.json',
 'CMIP6_Oday.json',
 'CMIP6_Odec.json',
 'CMIP6_Ofx.json',
 'CMIP6_Omon.json',
 'CMIP6_Oyr.json',
 'CMIP6_SIday.json',
 'CMIP6_SImon.json',
 'CMIP6_day.json',
 'CMIP6_fx.json']

In [10]:
data = dr.get_all_variable_entries_by_attributes(
    long_name="Near-Surface Air Temperature", frequency="mon", how="all"
)  # .keys()
data

{'CMIP6_Amon.json': {'tas': {'frequency': 'mon',
   'modeling_realm': 'atmos',
   'standard_name': 'air_temperature',
   'units': 'K',
   'cell_methods': 'area: time: mean',
   'cell_measures': 'area: areacella',
   'long_name': 'Near-Surface Air Temperature',
   'comment': 'near-surface (usually, 2 meter) air temperature',
   'dimensions': 'longitude latitude time height2m',
   'out_name': 'tas',
   'type': 'real',
   'positive': '',
   'valid_min': '',
   'valid_max': '',
   'ok_min_mean_abs': '',
   'ok_max_mean_abs': ''}},
 'CMIP6_ImonAnt.json': {'tas': {'frequency': 'mon',
   'modeling_realm': 'landIce land',
   'standard_name': 'air_temperature',
   'units': 'K',
   'cell_methods': 'area: time: mean where ice_sheet',
   'cell_measures': '',
   'long_name': 'Near-Surface Air Temperature',
   'comment': 'near-surface (usually, 2 meter) air temperature',
   'dimensions': 'time height2m',
   'out_name': 'tas',
   'type': 'real',
   'positive': '',
   'valid_min': '',
   'valid_max': 

In [11]:
data = dr.get_all_variable_entries_by_attributes(out_name="sftlaf")  # .keys()
data

{}

In [40]:
def variables_not_in_cmip6(df):
    cordex_vars = df["output variable name"].unique()
    results = []
    for v in cordex_vars:
        data = dr.get_all_variable_entries_by_attributes(out_name=v)
        if not data:
            results.append(v)
    return results

In [41]:
variables_not_in_cmip6(df)

sund
rsdsdir
mrfsos
mrfsol
ua1000
ua925
ua850
ua700
ua600
ua500
ua400
ua300
ua250
ua200
va1000
va925
va850
va700
va600
va500
va400
va300
va250
va200
ta1000
ta925
ta600
ta400
ta300
ta250
ta200
hus1000
hus925
hus700
hus600
hus500
hus400
hus300
hus250
hus200
zg925
zg850
zg700
zg600
zg400
zg300
zg250
zg200
wa1000
wa925
wa850
wa700
wa600
wa500
wa400
wa300
wa250
wa200
ua50m
ua150m
va50m
va150m
ta50m
hus50m
wsgsmax
clh
clm
cll
rluscs
z0
CAPE
LI
CIN
CAPEmax
LImax
CINmax
od550aer 
ua150
ua100
ua70
ua50
ua30
ua20
va150
va100
va70
va50
va30
va20
va10
ta150
ta100
ta70
ta50
ta30
ta20
ta10
hus150
hus100
hus70
hus50
hus30
hus20
hus10
zg150
zg70
zg50
zg30
zg20
wa150
wa100
wa70
wa50
wa30
wa20
wa10
ua750
va750
ta750
hus750
zg750
wa750
ua200m
ua250m
ua300m
va200m
va250m
va300m
sftlaf
sfturf
dtb


['sund',
 'rsdsdir',
 'mrfsos',
 'mrfsol',
 'ua1000',
 'ua925',
 'ua850',
 'ua700',
 'ua600',
 'ua500',
 'ua400',
 'ua300',
 'ua250',
 'ua200',
 'va1000',
 'va925',
 'va850',
 'va700',
 'va600',
 'va500',
 'va400',
 'va300',
 'va250',
 'va200',
 'ta1000',
 'ta925',
 'ta600',
 'ta400',
 'ta300',
 'ta250',
 'ta200',
 'hus1000',
 'hus925',
 'hus700',
 'hus600',
 'hus500',
 'hus400',
 'hus300',
 'hus250',
 'hus200',
 'zg925',
 'zg850',
 'zg700',
 'zg600',
 'zg400',
 'zg300',
 'zg250',
 'zg200',
 'wa1000',
 'wa925',
 'wa850',
 'wa700',
 'wa600',
 'wa500',
 'wa400',
 'wa300',
 'wa250',
 'wa200',
 'ua50m',
 'ua150m',
 'va50m',
 'va150m',
 'ta50m',
 'hus50m',
 'wsgsmax',
 'clh',
 'clm',
 'cll',
 'rluscs',
 'z0',
 'CAPE',
 'LI',
 'CIN',
 'CAPEmax',
 'LImax',
 'CINmax',
 'od550aer ',
 'ua150',
 'ua100',
 'ua70',
 'ua50',
 'ua30',
 'ua20',
 'va150',
 'va100',
 'va70',
 'va50',
 'va30',
 'va20',
 'va10',
 'ta150',
 'ta100',
 'ta70',
 'ta50',
 'ta30',
 'ta20',
 'ta10',
 'hus150',
 'hus100',
 'hus70